# Toggle switch

In [ ]:
from output_helper import *
%matplotlib inline

In [ ]:
with nc.Dataset("output/ts/output_t50000.nc", "r") as ds:
    grid = grid_info(ds)
    slice_vec = np.array([0, 0])
    P_full = fullDistribution(ds, grid)
    P_marginal = marginalDistributions(ds, grid)
    P_sliced = slicedDistributions(ds, grid, slice_vec)

In [ ]:
with open("scripts/reference_solutions/ts_ode_ref.npy", "rb") as f:
    P_full_ode = np.load(f, allow_pickle=True)
    P_marginal_ode = np.load(f, allow_pickle=True)
    P_sliced_ode = np.load(f, allow_pickle=True)
    P_best_approximation_ode = np.load(f, allow_pickle=True)
    n_ode = np.load(f)

Parameters for Figure 4:

In [ ]:
t = 500
time_step = 10
snapshot = 100

## Figure 1
One-dimensional marginal distributions for $t = 500$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2))
grid_ref = [range(el.shape[0]) for el in P_marginal_ode[-1]]
axs = plotP1Dmult(
    axs, P_marginal, P_marginal_ode[-1], grid, grid_ref, ["DLR approx.", "exact"], range(2), "MD")
axs.flat[0].legend()
fig.tight_layout()
fig.savefig('plots/ts_fig1.pgf')

## Figure 2
Sliced distributions for $x_0 = (0,\,0)$, $t = 500$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2))
grid_ref = [range(el.shape[0]) for el in P_sliced_ode[-1]]
axs = plotP1Dmult(
    axs, P_sliced, P_sliced_ode[-1], grid, grid_ref, ["DLR approx.", "exact"], range(2), "S")
fig.tight_layout()
fig.savefig('plots/ts_fig2.pgf')

## Figure 3
Full probability distribution for $t = 500$

In [ ]:
xx1, xx2 = np.meshgrid(
    np.arange(grid.n1[0]), np.arange(grid.n2[0]), indexing='ij')
fig, ax = plotP2D(
    P_full, P_full_ode[-1], [xx1, xx2], ["DLR approximation", "exact"])
plt.setp(ax.flat, xlim=[0, 30], ylim=[0, 30], aspect="equal")
fig.tight_layout()
fig.savefig('plots/ts_fig3.pgf')

Maximal error for the full probability distribution:

In [ ]:
print(np.amax(P_full - P_full_ode[-1]))

## Figure 4
Calculate the 2-norm error for the DLR approximation and for the truncated SVD of the reference solution ('best approximation')

In [ ]:
times = np.arange(0, t + time_step, time_step)
err = np.zeros(times.size)
errba = np.zeros(times.size)

for ts, tp in enumerate(times):
    filename = "output/ts/output_t" + str(int(tp * snapshot)) + ".nc"
    with nc.Dataset(filename, "r") as ds:
        P = fullDistribution(ds, grid)
    err[ts] = np.linalg.norm(P - P_full_ode[ts], ord=2)
    errba[ts] = np.linalg.norm(
        P_best_approximation_ode[ts] - P_full_ode[ts], ord=2)

fig, ax = plt.subplots(figsize=(5, 3))
plotP1D(ax, err, errba, times, [
        "DLR approximation", "best-approximation"], xlabel="$t$", ylabel="2-norm error")
ax.legend()
fig.tight_layout()
fig.savefig('plots/ts_fig4.pgf')